<h1>Introduction to Reinforcement Learning with TensorFlow</h1>
<p>This tutorial was adapted from various <a href="https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724">examples</a> provided by Arthur Juliani. I would recommend their various blog posts for further reading on Reinforcement Learning.</p>
<p>The following notebook introduces some of the key concepts of Reinforcment Learning with Python and TensorFlow. Please make sure that the following import statements run successfully before continuing:</p>

In [1]:
import tensorflow as tf
import gym
import numpy as np

/home/student/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


<p>Now that we have this taken care of, we can begin  with a brief introduction to one subset of Reinforcement Learning methods: Policy based methods. Policy based methods allow us to make our agent improve its performance by determining which policies for acting produce better results. This is different from more state based methods referred to as Q learning methods. <a href="https://flyyufelix.github.io/2017/10/12/dqn-vs-pg.html">This writeup</a> provides a relatively concise comparision of the two methods, but in short: policy methods seek to act optimally while Q learning methods seek to reach optimal states. Both methods may be used to make an intelligent agent, but for the examples we are working with here, our state space is more continuous than discrete.</p>
<p>Our agent should operate by processing its current <i>observation</i> of the world to choose <i>actions</i> which maximize a <i>reward</i> feedback symbol. The specific world our agent will work in is referred to as the <i>environment</i>, and for this example, we will be working with OpenAI's <a href="https://gym.openai.com/docs/">Cart-Pole</a> environment. Running the following code should set up our example environment.</p>

In [2]:
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


<p>Now that we have our environment set up, we can take a look at what it looks like using the render() function. For this environment, our goal is to train an agent to balance a pivoting pole on a rolling cart. The following code will reset our environment and use an agent which randomly performs actions for 20 episodes, each composed of 100 frames.</p>

<p>Note: if the following code fails, try downgrading the pyglet package using the following command:</p>
<div style="background-color:#300a24"><b><p style="color:white">python3 -m pip install pyglet==1.2.4</p></b></div>

In [3]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

Episode finished after 14 timesteps
Episode finished after 12 timesteps
Episode finished after 13 timesteps
Episode finished after 25 timesteps
Episode finished after 18 timesteps
Episode finished after 12 timesteps
Episode finished after 24 timesteps
Episode finished after 10 timesteps
Episode finished after 38 timesteps
Episode finished after 19 timesteps
Episode finished after 10 timesteps
Episode finished after 16 timesteps
Episode finished after 16 timesteps
Episode finished after 18 timesteps
Episode finished after 20 timesteps
Episode finished after 11 timesteps
Episode finished after 35 timesteps
Episode finished after 12 timesteps
Episode finished after 32 timesteps
Episode finished after 23 timesteps


<h1>Observation Space</h1>
<p>It is important to note what our observation space is. Our observations represent how our agent perceives the world, and in the case of the CartPole environment, our observations consist of 4 numbers representing the position of the cart, the velocity of the cart, the angle of the pole, and the rotational velocity of the pole. Additional information about the CartPole environment can be found <a href="https://github.com/openai/gym/wiki/CartPole-v0">here</a>.</p>

In [4]:
print('Observation space:', env.observation_space)

Observation space: Box(4,)


<h1>Action Space</h1>
<p>The other key aspect our AI needs to understand is the action space, which represents what choices our agent has each timestep. In this example, the cart has only 2 options: push the cart left or push the cart right.</p>

In [5]:
print('Action space:', env.action_space)

Action space: Discrete(2)


<h1>Rewards</h1>
<p>Now that we know what the world looks like and what our options are, we need to understand how we rate our agent's performance. In this example, the agent receives a reward of 1 point for each timestep. The episode will continue to give the agent rewards until either of the following occur:</p>
<ol>
<li>The cart moves too far left or right off screen.</li>
<li>The pole falls over.</li>
<li>The episode reaches 200 timesteps.</li>
</ol>
<p>As a result, we should expect that an intelligent agent will work to stay on screen while keeping the pole balanced upright for as long as it can (up to 200 timesteps). In order to optimally train our agent, we train the agent using to maximize its expected rewards by backpropogating discounted rewards. This means that after each episode, we look back at our experience and add our final reward discounted by our parameter gamma ($\gamma$).</p>
<p>Gamma can be interpretted as how far/near -sighted our agent is. With a value close to 0, our agent does not care as much about future rewards. With a value close to 1, our agent cares more about future rewards. Typically, you would want a value close to, but not necessarily 1. For this example, we choose $\gamma=0.99$.</p>

In [6]:
gamma = 0.99

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

<h1>Making the Agent</h1>
<p>With these abstract considerations, we can start thinking about how we want our agent to work. Each frame, our agent will receive 4 numerical inputs and produce one of two possible outputs. We can think of our agent as a function of 4 inputs which produces a singular output:</p>
$$f(x_1,x_2,x_3,x_4) \to y$$
<p>We will now start working on implementing our RL agent. We will define a class for our agent which will allow us to define the size of our input (our state information) and the size of our output (our action choice) so that we can more easily adapt this code for later projects. Our constructor begins by defining a set of TensorFlow placeholders for each input. From there, we feed everything into a dense hidden layer of neurons. The neural network architecture for this agent is illustrated below:</p>
<img src="Neural_Network_CartPole.png"></img>

In [7]:
class Agent:
    #Agent constructor to allow us to change the state/action size
    def __init__(self, state_size, action_size, hidden_size=8):
        #Create placeholders for passing in our state information
        self.input = tf.placeholder(shape=[None, state_size], dtype=tf.float32)
        #Connect our input to the first, hidden layer
        self.hidden_layer = tf.layers.dense(self.input, hidden_size, activation=tf.nn.relu)
        #Connect our hidden layer to our output layer for our 2 actions
        self.output = tf.layers.dense(self.hidden_layer, action_size, activation=None)
        self.softmax_output = tf.nn.softmax(self.output)        #Determine which action (0 or 1) is picked by picking the max output node
        self.choice = tf.argmax(self.softmax_output, 1)

#These last few lines delete any existing TensorFlow variables and create an instance of our agent.
tf.reset_default_graph()
my_agent = Agent(state_size=4, action_size=2)
#print out the number of trainable variables for our agent (for visualization purposes only)
def calculate_trainable_variables():
    trainable_vars = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        variable_parameters = 1
        for dimension in shape:
            variable_parameters *= dimension.value
        trainable_vars += variable_parameters
    return trainable_vars
print('Number of trainable variables for our agent:', calculate_trainable_variables())

Number of trainable variables for our agent: 58


<p>The above code sets up our agent's neural network like the diagram shown above, but it is missing a key part:<br>
the actual the ability to train the network and change it's variables! We will explore one way in which we might accomplish this. In total, for our agent with just one hidden layer of 8 neurons, 4 input values, and an output layer for 2 actions, our agent has over 50 unique parameters to train. As the state, action, and hidden layer sizes increase, the number of trainable parameters increases dramatically, so it is important that we are training our agent efficiently.</p>

<p>When many people think of how they can improve the performance of an AI, genetic algorithms often come to mind. For this method, we would ideally take the best performing agents and tweak their values slightly. For this example, we would take two agents and then make a random selection of half of one agent's variables with those of another as our child agent. This agent will thus be based on both of its two parent agents' parameters. From here, we would mutate a random subset of the child agent's parameters to ensure that our agents do not immediately converge to a suboptimal agent. In theory, this could work, but there are more reliable methods for improvement which we will cover here.</p>

<p>In Reinforcement Learning, we can take advantage of Calculus to make more educated guesses about how to improve our AI. The general idea is that we can quantify our performance as a numerical score, and by comparing this score with previous scores, we can determine our <i>loss</i> function. Using multivariable calculus, we can approximate how much each of our 50+ variables contributed to our loss function changing, which gives us insight on how we can change our AI's parameters!</p>

<p>In order to make this happen, we will need to add a placeholder for the reward our agent earned as well as which action the agent picked. With this, the agent should be able to reflect back on the long term consequences of its actions. We will make a new class extending the Agent class so we do not have to rewrite the previous code to add the reward/loss calculations.  (For more information about some of the specific TensorFlow functions such as the softmax_cross_entropy_with_logits, check the <a href="https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits_v2">documentation</a>)</p>

In [8]:
class TrainableAgent(Agent):
    def __init__(self, state_size, action_size, hidden_size=8,
                 optimizer=tf.train.AdamOptimizer(learning_rate=1e-2)):
        super().__init__(state_size, action_size, hidden_size)
        #Add the additional placeholders for reward and chosen action
        self.reward = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_chosen = tf.placeholder(shape=[None], dtype=tf.int32)
        #Create a vector to represent the chosen action ([1, 0] for 0 and [0, 1] for 1)
        self.one_hot_actions = tf.one_hot(self.action_chosen, action_size)
        #Determine how much each action probability changed based on our previous output layer
        self.cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.output,
                                                                        labels=self.one_hot_actions)
        #Determine our final loss with respect to how chaotic our action choices/rewards are
        self.loss = tf.reduce_mean(self.cross_entropy * self.reward)
        #Create a TensorFlow function to optimize our agent's performance with respect to loss
        variables = tf.trainable_variables()
        self.gradient_placeholders = [] #gradients represent how much each variable should change
        for index, variable in enumerate(variables): #read over each variable we can train
            #create a placeholder for the gradient to be applied
            placeholder = tf.placeholder(dtype=tf.float32)
            self.gradient_placeholders.append(placeholder)
        #Now that we have placeholders for all of the variables we can change,
        #we create a shorthand function to calculate the gradients with respect to 
        self.gradients = tf.gradients(self.loss, variables)
        #use the gradient optimizer passed in to optimize our agent
        self.update = optimizer.apply_gradients(zip(self.gradient_placeholders, variables))
        

<h1>Training the Agent</h1>
<p>Now that we have created our agent and added gradient optimization for its parameters, we need to train our agent. When we train our agent, there are a few things to consider:</p>
<ol>
<li>How many episodes are we going to train our agent on.</li>
<li>How long we will allow each episode to last.</li>
<li>How frequently will we update our neural network.</li>
<li>How frequently will we save our network variables to a file.</li>
</ol>

In [9]:
total_episodes = 1000
max_steps_per_episode = 500
update_frequency = 5 #every 5 episodes
save_frequency = 100
tf.reset_default_graph()
my_agent = TrainableAgent(state_size=4, action_size=2)
print('Number of trainable variables for our agent:', calculate_trainable_variables())

init = tf.global_variables_initializer() #we will use this to reset our agent's variables at the start

Number of trainable variables for our agent: 58


<p>Now we can go ahead and continue working on implementing our training loop. Here, we will run an interactive TensorFlow session and simulate our CartPole environment repeatedly for our agent. After every 5 episodes, we will update the agent's neural network variables with the gradient optimization we implemented earlier.</p>

In [17]:

with tf.Session() as sess:
    #initialize all of our variables
    sess.run(init)
    total_rewards = []
    total_length = []
    gradient_buffer = sess.run(tf.trainable_variables())
    for ix,grad in enumerate(gradient_buffer):
        gradient_buffer[ix] = grad * 0
        
    for episode in range(total_episodes):
        state = env.reset()
        running_reward = 0
        episode_history = []
        for step in range(max_steps_per_episode):
            #Probabilistically pick an action given our network outputs.
            action_distribution = sess.run(my_agent.softmax_output,feed_dict={my_agent.input:[state]})
            action = np.random.choice(action_distribution[0],p=action_distribution[0])
            action = np.argmax(action_distribution == action)

            state_next, reward, done, info = env.step(action) #Get our reward for taking an action given a bandit.
            episode_history.append([state, action, reward, state_next])
            state = state_next
            running_reward += reward
            if done:
                #Update the network.
                episode_history = np.array(episode_history)
                episode_history[:,2] = discount_rewards(episode_history[:,2])
                feed_dict={my_agent.reward : episode_history[:,2],
                        my_agent.action_chosen : episode_history[:,1],
                           my_agent.input : np.vstack(episode_history[:,0])}
                grads = sess.run(my_agent.gradients, feed_dict=feed_dict)
                for idx,grad in enumerate(grads):
                    gradient_buffer[idx] += grad

                if episode % update_frequency == 0 and episode > 0:
                    feed_dict = dictionary = dict(zip(my_agent.gradient_placeholders, gradient_buffer))
                    _ = sess.run(my_agent.update, feed_dict=feed_dict)
                    for ix,grad in enumerate(gradient_buffer):
                        gradient_buffer[ix] = grad * 0
                
                total_rewards.append(running_reward)
                total_length.append(step)
                break

        
            #Update our running tally of scores.
        if episode % save_frequency == 0:
            #TO DO: SAVE MODEL
            print(np.mean(total_rewards[-save_frequency:]))
            
    print('Training finished.')
    #TO DO: SAVE MODEL

9.0
15.08
9.74
9.3
9.5
9.38
9.43
9.37
9.31
9.34
Training finished.


<h1>Saving our Results</h1>
<p>With our agent now trained, we can now save the neural network weights to a file using TensorFlow's built-in Saver class. The code below will create a Saver object with a maximum of 2 checkpoints saved for our agent and save it to the directory labeled 'cartpole-pg'.</p>

In [16]:
import os #for working with files/folders

saver = tf.train.Saver(max_to_keep=2)
folder_path = './cartpole-pg/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    


<p>After we have made our saver object, we can use it to save our current neural network so we can load it later. The code below will save our model to the folder we made, but we must only run this code inside our training block above.</p>

In [ ]:
#Place this line  in the training code above so that our agent saves its progress periodically and at the end
saver.save(sess, folder_path + 'pg-checkpoint', episode)

<h1>Testing our Agent</h1>
<p>Now that we have created and trained our agent, we will test the agent on a series of episodes to evaluate its performance ourselfs. The code below will load our previously saved session data into the agent.</p>

In [21]:
testing_episodes = 5

with tf.Session() as sess:
    #check our folder for saved checlpoints
    checkpoint = tf.train.get_checkpoint_state(folder_path)
    #restore the checkpoint for our agent
    saver.restore(sess, checkpoint.model_checkpoint_path)
    #Run our agent through a series of testing episodes
    for episode in range(testing_episodes):
            state = env.reset()
            episode_rewards = 0
            for step in range(max_steps_per_episode):
                env.render()
                # Get Action
                action_argmax = sess.run(my_agent.choice, feed_dict={my_agent.input: [state]})
                action_choice = action_argmax[0]
                 
                state_next, reward, done, _ = env.step(action_choice)
                state = state_next
                 
                episode_rewards += reward
                 
                if done or step + 1 >= max_steps_per_episode:
                    print("Rewards for episode " + str(episode) + ": " + str(episode_rewards))
                    break

INFO:tensorflow:Restoring parameters from ./cartpole-pg/pg-checkpoint-999
Rewards for episode 0: 10.0
Rewards for episode 1: 10.0
Rewards for episode 2: 9.0
Rewards for episode 3: 9.0
Rewards for episode 4: 9.0


<h1>Results</h1>
<p>What kind of results do you get with your agent? Is your agent reaching average scores of 200? What changes might you make to the agent's neural network structure, the number of training episodes, learning rate, and other parameters to increase its perofrmance?</p>